In [ ]:
from abc import ABC, abstractmethod
import pandas as pd
from typing import List, Dict, Any

class OpenFoodFactsProduct:
    def __init__(self, code: str, df: pd.DataFrame) -> None:
        self.code = code
        self.product = df.loc[df['code'] == code].copy()

    def get_category(self) -> str:
        if self.product.empty:
            raise ValueError(f"No product found with code {self.code}")
            
        if self.product["categories_en"].str.contains("beverages").any():
            return "beverage"
        return "solid"

In [ ]:
class NutritionalRatingSystem(ABC):
    def __init__(self, name="nutriscore", required_factors=["energy_100g", "energy_kcal_100g", "proteins_100g", "fiber_100g", "saturated-fat_100g", "sugars_100g", "fruits-vegetables-nuts-estimate-from-ingredients_100g ", "salt_100g"]):
        self.name = name,
        self.required_factors = required_factors
    
    @abstractmethod
    def calculate_score(self, product_code):
        pass
    
    @abstractmethod
    def rate(self, product_code):
        pass

In [ ]:
class Nutriscore(NutritionalRatingSystem):
    def __init__(self):
        super().__init__()
    

     # Progi punktowe dla różnych kategorii
        self.ENERGY_THRESHOLDS = {
            "solid": [80, 160, 240, 320, 400, 480, 560, 640, 720, 800],
            "beverage": [7.2, 14.3, 21.5, 28.5, 35.9, 43.0, 50.2, 57.4, 64.5, float('inf')]
        }
        
        self.SUGAR_THRESHOLDS = {
            "solid": [4.5, 9, 13.5, 18, 22.5, 27, 31, 36, 40, 45],
            "beverage": [0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0, 10.5, 12.0, 13.5]
        }
        
        self.SATURATED_FAT_THRESHOLDS = {
            "cooking_fats": [10, 16, 22, 28, 34, 40, 46, 52, 58, 64],
            "default": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        }
        
        self.SODIUM_THRESHOLDS = [90, 180, 270, 360, 450, 540, 630, 720, 810, 900]
        self.FIBER_THRESHOLDS = [0.7, 1.4, 2.1, 2.8, 3.5]
        self.PROTEIN_THRESHOLDS = [1.6, 3.2, 4.8, 6.4, 8.0]

    def _score_based_on_thresholds(self, value, thresholds, max_score):
        """Pomocnicza metoda do obliczania punktów na podstawie progów"""
        for i, threshold in enumerate(thresholds):
            if value <= threshold:
                return i
        return max_score

    def calculate_score(self, product: OpenFoodFactsProduct):
        if product.empty:
            raise ValueError("Product not found")
        
        product_category = product.get_category()
        
        # Punkty negatywne
        negative_points = sum([
            self.score_energy(product["energy_100g"].values[0], product_category),
            self.score_sugars(product["sugars_100g"].values[0], product_category),
            self.score_saturated_fat(product["saturated-fat_100g"].values[0], product_category),
            self.score_sodium(product["salt_100g"].values[0], product_category)
        ])
        
        # Punkty pozytywne
        positive_points = sum([
            self.score_fiber(product["fiber_100g"].values[0]),
            self.score_protein(product["proteins_100g"].values[0]),
            self.score_fruits_vegetables_nuts(
                product["fruits-vegetables-nuts-estimate-from-ingredients_100g"].values[0],
                product_category
            )
        ])
        
        return negative_points - positive_points

    def score_energy(self, energy, product_category):
        thresholds = self.ENERGY_THRESHOLDS[product_category]
        return self._score_based_on_thresholds(energy, thresholds, 10)

    def score_sugars(self, sugars, product_category):
        thresholds = self.SUGAR_THRESHOLDS[product_category]
        return self._score_based_on_thresholds(sugars, thresholds, 10)

    def score_saturated_fat(self, saturated_fat, product_category):
        category = "cooking_fats" if product_category == "cooking_fats" else "default"
        thresholds = self.SATURATED_FAT_THRESHOLDS[category]
        return self._score_based_on_thresholds(saturated_fat, thresholds, 10)

    def score_sodium(self, sodium, product_category=None):
        return self._score_based_on_thresholds(sodium, self.SODIUM_THRESHOLDS, 10)

    def score_fiber(self, fiber):
        return self._score_based_on_thresholds(fiber, self.FIBER_THRESHOLDS, 5)

    def score_protein(self, protein):
        return self._score_based_on_thresholds(protein, self.PROTEIN_THRESHOLDS, 5)

    def score_fruits_vegetables_nuts(self, fruits_vegetables_nuts, product_category):
        if product_category == "beverage":
            if fruits_vegetables_nuts <= 40:
                return 0
            elif fruits_vegetables_nuts <= 60:
                return 2
            elif fruits_vegetables_nuts <= 80:
                return 4
            return 10
        else:
            if fruits_vegetables_nuts <= 40:
                return 0
            elif fruits_vegetables_nuts <= 60:
                return 1
            elif fruits_vegetables_nuts <= 80:
                return 2
            return 5

    def rate(self, product: OpenFoodFactsProduct):
        score = self.calculate_score(product)
        product_category = product.get_category()
        
        if product_category == "solid":
            if score <= -1:
                return "A"
            elif score <= 2:
                return "B"
            elif score <= 10:
                return "C"
            elif score <= 18:
                return "D"
            return "E"
        elif product_category == "beverage":
            if score == 0:
                return "A"
            elif score <= 1:
                return "B"
            elif score <= 5:
                return "C"
            elif score <= 9:
                return "D"
            return "E"

In [ ]:
class RecommendationStrategy():
    def __init__(self, nutritional_rating_systems: List[NutritionalRatingSystem], tags_to_avoid: List[str], allergens_to_avoid: List[str]) -> None:
        self.nutritional_rating_systems = nutritional_rating_systems
        self.tags_to_avoid = tags_to_avoid
        self.allergens_to_avoid = allergens_to_avoid